The Traveling Salesman Problem (TSP) is a classic combinatorial optimization problem that seeks the shortest possible route visiting a set of locations exactly once and returning to the starting point.
 It is formally defined as finding the shortest closed path that visits each vertex in a graph exactly once, where the graph is typically undirected and weighted, with vertices representing cities and edge weights representing distances or costs.
 The problem is NP-hard, meaning no known polynomial-time algorithm can solve all instances optimally, and the number of possible routes grows factorially with the number of cities, making brute-force solutions impractical for large instances 

In [111]:
from itertools import combinations
import numpy as np

## Simple Test Problem

## Common tests

In [112]:
def evaluate(solution, problem):
    total_distance = 0.0
    for i in range(len(solution)):
        total_distance += problem[solution[i - 1], solution[i]]
    return total_distance

def mutate(population):
    """Apply 2-opt mutation to a randomly selected individual."""
    individual = population[np.random.randint(len(population))].copy()
    idx1, idx2 = np.sort(np.random.choice(len(individual), 2, replace=False))
    individual[idx1:idx2] = individual[idx1:idx2][::-1]
    return individual

def select_parents(population, fitness, tournament_size=3):
    """Tournament selection: works with any fitness values, including negative."""
    parents = []
    for _ in range(2):
        # Randomly select tournament_size individuals
        tournament_indices = np.random.choice(len(population), size=tournament_size, replace=False)
        # Select the one with lowest fitness (best for minimization)
        best_idx = tournament_indices[np.argmin(fitness[tournament_indices])]
        parents.append(best_idx)
    return parents

def crossover(parents):
    """
    Single-inversion crossover based on Inver-Over operator.
    Performs exactly one edge-guided inversion.
    """
    parent1, parent2 = parents[0].copy(), parents[1].copy()
    child = parent1.copy()
    n = len(child)
    
    # Select random starting city position
    c_idx = np.random.randint(0, n)
    c = child[c_idx]
    
    # Find c in parent2 and get its successor
    c_pos_in_parent2 = np.where(parent2 == c)[0][0]
    selected_city = parent2[(c_pos_in_parent2 + 1) % n]
    
    # Find position of selected city in child
    selected_idx = np.where(child == selected_city)[0][0]
    
    # Perform inversion between c and selected_city
    if c_idx < selected_idx:
        # Simple case: no wraparound
        child[c_idx+1:selected_idx+1] = child[c_idx+1:selected_idx+1][::-1]
    else:
        # Wraparound case: segment crosses the array boundary
        segment = np.concatenate([child[c_idx+1:], child[:selected_idx+1]])
        segment = segment[::-1]
        len_right = n - c_idx - 1
        child[c_idx+1:] = segment[:len_right]
        child[:selected_idx+1] = segment[len_right:]
    
    return child

def create_solution(problem):
    #Evolutionary algorithm
    population_size = 100
    generations = 500
    crossover_rate = 0.9
    
    # Initialize population
    population = [np.random.permutation(problem.shape[0]) for _ in range(population_size)]
    
    # Find initial best solution with positive fitness
    best_solution = None
    best_length = float('inf')
    for ind in population:
        fitness = evaluate(ind, problem)
        if fitness > 0 and fitness < best_length:
            best_solution = ind.copy() 
            best_length = fitness
    
    # If no positive solution found initially, use any solution
    if best_solution is None:
        best_solution = population[0].copy()
        best_length = evaluate(best_solution, problem)

    for generation in range(generations):
        # Evaluate fitness
        fitness = np.array([evaluate(individual, problem) for individual in population])
        
        # Create new population
        new_population = []
        for _ in range(population_size):
            if np.random.rand() < crossover_rate:
                # Crossover
                parents = select_parents(population, fitness)
                offspring = crossover([population[parents[0]], population[parents[1]]])
            else:
                # Mutation
                offspring = mutate(population)
            # Add to new population
            new_population.append(offspring)
        
        # Elitist selection: combine old and new populations, keep best
        combined_population = population + new_population
        combined_fitness = [evaluate(ind, problem) for ind in combined_population]
        # Sort indices by fitness (lower is better)
        sorted_indices = np.argsort(combined_fitness)
        # Keep the best population_size individuals
        population = [combined_population[i] for i in sorted_indices[:population_size]]
        
        # Update best solution (only consider positive fitness)
        for ind in population:
            current_fitness = evaluate(ind, problem)
            if current_fitness > 0 and current_fitness < best_length:
                best_solution = ind.copy()
                best_length = current_fitness
        
        #gradually decrease rate of crossover, last generation has around 10% chance of crossover
        crossover_rate = crossover_rate*0.995 
    print("Best distance:", best_length)
    return best_solution


In [113]:

problem = np.load('lab2/problem_r1_10.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 188.60215727417992
Solution: [5 6 2 9 7 1 8 0 3 4]


In [114]:

problem = np.load('lab2/problem_r1_10.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 194.379751542031
Solution: [0 4 5 6 2 9 7 1 8 3]


In [115]:

problem = np.load('lab2/problem_r2_100.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 4.08308843261694
Solution: [50 15  7 51 91 59 28 89 94 70 78 86 75  9 32 26 33 53 20 16 88 85 36  2
 22 56 66 61 87 12 46 23  1 68 57 55 11 41 98 47 84 58 67 65 54  3 17 52
 92 48 39  8 71 69 97 37 99 35 76 81 40 82 90 18 44 79 19 30 77  4 45 21
 49  6 43  5 72 10 31 29 25 34 62 80 96 74  0 63 13 83 42 95 60 73 24 27
 14 38 93 64]


In [116]:

problem = np.load('lab2/problem_g_10.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 1497.6636482252907
Solution: [4 5 9 7 0 8 2 3 1 6]


In [117]:

problem = np.load('lab2/problem_g_20.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 1755.5146770830047
Solution: [17  2  9 12  7  1 16 14 19 15  3  8  6 13 11  0  5 18 10  4]


In [118]:

problem = np.load('lab2/problem_g_50.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 2858.271343810905
Solution: [43  4  2  9 19 29 32 42 24 31 20 26 39 37 21 33 36 41  1 12 49 38  8 17
 48 40 28 15 13 35 16 45 44 23 30  5 25 27  0 14 18 46 47  7  6 34 10  3
 11 22]


In [119]:

problem = np.load('lab2/problem_g_100.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 5567.269951992501
Solution: [31 10 97 30 81 68 89 77 85 83 52 56 57 18 70  2 73 58 11 60 28 21 51  3
 84 66 45 53 79 17 44 94 16 38 95 35 61 23 62  9 59 82 39 98 25 19 80 65
 29 76 90 13 96 26 50 15 24 40 91 12 43 87 88 78 92  6 34 75 86 55 93 46
 36  1 69 22  4  8  5 99 20 27 63 47 71 74  7 49 37 32 48 72 14  0 33 41
 54 67 42 64]


In [120]:

problem = np.load('lab2/problem_g_200.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 17158.426494567007
Solution: [ 67  59  62  42 167  26  36 161  72 105  15  70 177 187  47 190 160 184
 116 107 186 126 196  22 101 151 119  23 189  58  79  28 147  18  39  89
 141  73 121 168 182  13  54  69  25 145  50 148 117 115  98 154 194 172
 106  56 156 173  82 123  71 100 128 125 159 153  11  43 178 139  46 175
  21  68 103 104 158 130  48 157   2 144  41  40  57 132  10 133 142  53
 110  33 165  81  27 102 176  78 111 155 170 124  77  66  17   1  74 146
 185  97 140  85 131 143  52  51  86 199  93  31   5 118  80 166  29  91
   9  38  92  87 120  12  84  37 137  16  45  94 127 114   3 183  55  34
  88   7  64   0   8  30 150  65 152  49  61 122  83 180 129 179 134 135
  95  96  14 164 191 109  44   4 149  32 188 113  20  63  90 198  60 174
  24 136   6 163 195  35 108 192 162  75  19 169 193 138 112  76  99 181
 171 197]


In [121]:

problem = np.load('lab2/problem_g_500.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 72341.09354890444
Solution: [131 355 197  76 134 170 172 188 318 298 167  73 490 390 177 385  31 398
 283 151  23 451 173 258 343 453 389 165 422 184  96 193 263 405 204 498
 187 477 154 190 350 365  44 285 112  91 292 149 252 400 164  92 311 308
  54 123 178 487 415   2 133  18 348 305 273 100 448 286 495 437  86 207
 194 339  68 312  78  63 105 145 101 226 111 432 472 297 225 237 104  43
  70 341 272 147 176 474 334 443 484 248 218 161 468 480 135 148 214 166
 433 141 120 403 247 136 290 423 296 256 373 442 180 222 299 183 469 332
 235  47 315 360 492 257 234 370 327 464 434 396 407 244  81  57 425 129
 485 378 246 216 478 381 291 239 371 466 275  45   0 328 228 232 269  82
  62  88 444 346 179 213 340  79 267  28 331 317  11 242 336   7  29 261
 265 363 259 392 455 309 300  67 380 199 421 171 230 368 107 374 245 279
 254  15  38 114 320 414 388 116  98  35 160 195 219 401 397 115 250 139
 142 375 413 325 427 128 103 499 306 211 491   3 387 416  95 459 354 198
 393 323

In [122]:

problem = np.load('lab2/problem_g_1000.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 183993.53812851125
Solution: [689 854 428 241 518 736 278 851 305 798  58 232 911 527 922 585 875 168
 511 113  62 202 205 366 150 790 513 300 711 178 581 931 969 835  69 760
  89 750 677 727 174 721 240  41 775 517 680 956 637 495 319 337 564 117
 346 454  56 255  83 280 648 845  68  84 841 975 882 776 355 813 601  44
  30  97 799 551 364 662  82 639 396 190 392 658 520 234 958 833  96 236
 175 341 726 286 919 962 603 413 397 861  80 531  18 604 492 537 636 643
 830 578 932 754 317 616 374 173 844   5 133 770 999 557 809 535 777 368
  66 837 656 674  17 566 253 227 211 291 104 124 429 973  92 359 608 972
 549 342 450  71 852 751 188 664 659 902 640 780 340 250 961 385 322 207
 929 907 169 661 267 735 400 864 678 282 275 895 724 788  31  65 981 266
  15 357 737 295 121 287 862 984 652 957 983 829 803 225 217 951 607 136
 785 409  52 618 834 839 530   0 418 657 979 686 797 416 569 946 991 376
 910 108 763 310 420 663 264 909 696 787 914 884 729 687  51 394 709 960
 540 49

In [123]:

problem = np.load('lab2/problem_r1_20.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 436.2388464175467
Solution: [14 18  2  0  1  8 17 15  7 13  6 11 10  5  4  9 16 12  3 19]


In [124]:

problem = np.load('lab2/problem_r1_50.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 971.4879399864595
Solution: [42 34 16  1 38 35 41 18  5 12 19 47 22 10 11 33 31 28  4 15  8 21 36 39
 44 32 37  2 24 14 23  7 49 46  3  0 29 48 43 25 13 17 40 20 30  6  9 45
 26 27]


In [125]:

problem = np.load('lab2/problem_r1_200.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 5977.668200649353
Solution: [ 67  10  27  31  11  61  44 191  88  91  39 178 159 189  83 135 196 175
 147  26  98 127 160 158  33 121 163 104 132  54  58 157  48 101 190 119
 140  68 199 114  13 176 134 137 167  87  63 184  72  69 187  15  62  65
  71  30 156 161  29  22   7  96 194  80  84   1 169  66 138 144  32   4
 182  49  85  34 107  92  89 115 108  81   0 149 142 171 124 103  12  51
 118  38 183 131 193  14  76 168  74  73 195 153  64  25 172 155  40  24
  47 123 125  36 120 146 113  94   8 116  75  78  90  17 185 148   3 100
  18 186 150  79  52 133 179  21 145  37 174  56  97 122  57 143   9  55
 162 181 152   6 192 139  42  46 165  19 130  50  41 166 180 197  59  60
 141 117 151  35  23   5 110  45 129  95 106 111  43   2 109  86 188  16
  77 164 112 128  28 177 154  53 105  93  70 170 126  20  99 173 136 102
  82 198]


In [126]:

problem = np.load('lab2/problem_r1_500.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 19158.506892139092
Solution: [320 336 238 115 257 112 248 457 207 301  84 410 311 396 227 434 458 471
 341 463 478  74 424 349 105 203  41 476 293 259 229   2 174 161 468 273
 444 147 447 177 366 110  21 220 183 411 186 324 244  56 289 325 469  80
 121  11 100 354 265  10 188 163 343 489 243 454 128 173  32 400 302 480
   1  54 499 205  20 196  34 441 125 492 103 315  24 445 232 436 448  17
  96 284 281 242 153 385 347 286 364 312  45  46 386 256 481 234  12 431
 384  57 379 223 494 392 178 106 107 158 258  82 440 102 466 318 355  35
 351 299 442 269 155 470 333 181 137 482 423 251  76 402 486 252  68  33
  25 213 360 235 154 425  90 453 365 192 403 216  69 383   8 404 138 118
 280  86 314 459 329 142 182 390 473 132 415 477 249  67 271 165  70 382
 413  22 348 429 474 381 202 451 176  43 389 332 150 114  97 334 303 342
 326 487 339 319 226  60  71 126 290  28 438 418  15 206 461 133 369  55
 321 179 260 450 131 370  81 162 171  37 306 422 237 287 166 297 467   9
 340  8

In [127]:

problem = np.load('lab2/problem_r1_1000.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 43980.91875994447
Solution: [153 532 198 418  77 698  35 313 955 294 890 565 219 661 696 814 670 290
 210 782 697 208 887 928 427  21 926 387 130 676 207 980 112 386 947 988
 714 180 891  87  44 816 212 104 581 842 246 456 752 468 923  67 953 934
 490 500   0 193 603 957 998 892 919 587 822 138 790 165 292 470 199 535
 473 547 259  82 413 596 701 167 829 971  55 187 438 451 373 455 674 914
 452 912  90 345 469 467 792 423 857 949 284 402 794 394 728 765 330 920
 900 785 999  57 508 304 442 406 105 610 855 420 798 722 122  48 996 561
 812 329 233 372 789 550  47 192 391 831 600 504 619 271  84 183 509 285
  59 258 875 321 801 626 263 353 421 308 623 534 691 982 487 868 802 931
 458 625   6 653 147 896 462 526  66 499 737 878 946 863 828 665 377 251
 614 178 903 491 586 280 726 195 119 819 901 648 963 322 781 853 595 118
 551 608 461 783 629 821 441 906 448 759 117 516 266 700 289 606 956  25
 340 525  61 360 485 837 572 380 723 582 196 809 293 898 808 751 174 871
   1 466

In [128]:

problem = np.load('lab2/problem_r2_10.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 0.11669226188648452
Solution: [3 2 9 4 0 1 8 7 5 6]


In [129]:

problem = np.load('lab2/problem_r2_20.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 0.283131392545414
Solution: [16  9  0 15 18  6  4 17 13 12 19  5  8  3 11  2  1  7 14 10]


In [130]:

problem = np.load('lab2/problem_r2_50.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 5.226711716442281
Solution: [ 5  7 13 41 23 16  3 47 24 18 31 32  8 48  1 42 49  9  4 45 29 46 43 30
 11 35 10 34 40 26 17  6  2 44 12 39 25  0 14 20 38 15 21 36 28 22 27 19
 33 37]


In [131]:

problem = np.load('lab2/problem_r2_200.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 16.2491586438056
Solution: [ 39 108   1  44 156 150  69  96  10 138 186  45 117  49  19  93  63  73
  22  35 167  42 107  97 157  18 144 126 187  85 160 101 189 119  92  95
 174 181  47 100  59 188 143   5 158 154  64  14  57  33  90 112  29 183
  72  20 170   4 176  52  84 140 122  48 185 171 104 168 166 152  60  79
  70   3  36 169 190 114 155 133 116 124 103  38  37 159 151 173  61  56
   6  51  30 175 132  98  55 130 191 178 102 134 177  75 162  25 164  99
 137 135  86 136  11  27 127  26  82 193  31  50  65 142 109   9  43 105
  28 125  80 194 110  13  58  17 161 199  67 123  68  53  21  74  77  88
  41 184 120 113 118  66  91 106 198 197  34 148  81 115  89  23   0 149
  12 111 129 139 165  94   2  78  16 195  24 182 141  83  15 147 196 145
 163 131 146  46 121  32  62 153  40  76  54 172 192 180  87   7  71 128
 179   8]


In [132]:

problem = np.load('lab2/problem_r2_500.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 1.6027028060339603
Solution: [478 138 209 442 465 305  98 488  37 158 154  87 102 246 110 487 411 360
 157 135 104 399 220 283 134 304 486  34 225 420 375 171 329 356 396 239
 204  16 306 132  20 316 262 145 320 482  53 377 247 201 333 430 338 295
  58 116 492 440 311 446 441 457 477 349 380 337 196 190 124  41 160 115
  23 172 242 472 383 326 229 332 163  42 495  66  19  94 231 216 471 404
 385 224   1 412 434  47 105 435 227 455  89  71 221 335 485 318 426 278
  39 106 173 148 213  56 397 212 421 354 439 431 303 288  22 119 302 274
  36  48 366 118 357 347 100 253 267 494 459  24 355  90 287 346 369 232
 408  59  72 276 249 241 432 291  78 205   0  57 389  88 428 475 342 405
 460  84  15 181   8  62 281 206 210 250 178 463 260  43  52 444 371 307
 417 184 263 319 189 410 101 370 183 203 497 265  33 384 378 449   2 244
 300  60 168 197 286 325 297 322 498 339   6 406 496  73 185 230 199 330
   7 122 289 194 372  12 195 280 299 130 114 150 109 298 256 479 179   3
  55 23

In [133]:

problem = np.load('lab2/problem_r2_1000.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 16.919839310439393
Solution: [128 586 932 454 559 408 427 480 826 332 490 563 720 908   3 363 590 439
 313 699 641 315 247 430 677 607 496 185 800 362 804 520 353 725 936 768
 958 731 728 596 964 149 965 157 902 558 537 979 823  69 718 871 323  27
  43 239 123 822 115 351 608  91 378   0 918 141 988 819  12 942 803   9
 601 785 211  84 447 814 338 257   6 360 977 581 272 859 154 901 762 882
 509 743  22 542 110 305 887 946 821 593 459 612 179 610 689 367 155 922
 534 930 574 392 808 125  39 811 265 957 492  10 664 567 441 854 228 435
 764 956 912 647 667 306 970 815 213 397  45 594 652 288 634 428 830 249
 354 225 683   8 760  60 972 682 318 455 898 294 554 184 660 268 712 884
 178 369 250 617 401 560 990 996 650 614 191   5 317 761 783 524 673 862
 916 779 629 263 347 836 208 309 625 223 389 419 855 961 105 349 858 283
 997 416 522 312 659 284 452 866 657 393  15 905 874 955 395 873 810  61
 494 781 359  18 576 951 837  57 515 923 521 361 158  75 241  19 230 844
 118 75

In [134]:

problem = np.load('lab2/test_problem.npy')
solution = create_solution(problem)
print(f"Solution: {solution}")

Best distance: 2823.7899999999995
Solution: [19 18  5  3  1 16 11 13 10 14  8 15 12  9  4  2  0  7 17  6]
